In [62]:
import numpy
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

In [4]:
# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertIntermediate(
       

In [2]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

100%|██████████| 231508/231508 [00:00<00:00, 5798281.03B/s]


In [6]:
tokens_tensor = torch.tensor([indexed_tokens])

In [8]:
out = model(tokens_tensor)

In [9]:
out.size()

torch.Size([1, 11, 30522])

In [10]:
len(tokenized_text)

11

In [73]:
''' sequential generation '''

sample = True
max_len = 20
leed_out_len = 3 #max_len
random_future = False

seed_text = 'the meaning of life is'.split()
seed_len = len(seed_text)

init_text = seed_text + ['[MASK]'] * max_len
init_idx = tokenizer.convert_tokens_to_ids(init_text)
if random_future:
    for ii in range(max_len):
        init_idx[seed_len+ii] = numpy.random.randint(0, len(tokenizer.vocab))

for ii in range(max_len):
    out = model(torch.tensor([init_idx[:seed_len+ii+leed_out_len]]))
    if sample:
        dist = torch.distributions.categorical.Categorical(logits=out[0,seed_len+ii])
        init_idx[seed_len+ii] = dist.sample().item()
    else:
        init_idx[seed_len+ii] = torch.max(out[0, seed_len+ii],0)[1].item()

print(init_idx)
print(" ".join(tokenizer.convert_ids_to_tokens(init_idx)))
print(" ".join(tokenizer.convert_ids_to_tokens(init_idx)).replace(" ##", ""))

[1996, 3574, 1997, 2166, 2003, 2170, 2004, 26964, 3436, 2030, 2200, 19391, 1005, 1005, 1000, 1000, 1010, 1005, 1012, 2030, 1000, 5156, 2075, 1000, 2030]
the meaning of life is called as confronting ##ting or very emerges ' ' " " , ' . or " usual ##ing " or
the meaning of life is called as confrontingting or very emerges ' ' " " , ' . or " usualing " or


In [ ]:
''' parallel generation '''

sample = True
max_iter = 100
viz_int = 10
max_len = 20

seed_text = 'the meaning of life is'.split()
seed_len = len(seed_text)

init_text = seed_text + ['[MASK]'] * max_len
init_idx = tokenizer.convert_tokens_to_ids(init_text)
for ii in range(max_len):
    init_idx[seed_len+ii] = numpy.random.randint(0, len(tokenizer.vocab))

for ii in range(max_iter):
    out = model(torch.tensor([init_idx]))
    for kk in range(max_len):
        if sample:
            dist = torch.distributions.categorical.Categorical(logits=out[0,seed_len+kk])
            init_idx[seed_len+kk] = dist.sample().item()
        else:
            init_idx[seed_len+kk] = torch.max(out[0, seed_len+kk],0)[1].item()
    if numpy.mod(ii, viz_int) == 0:
        print("iter", ii+1, " ".join(tokenizer.convert_ids_to_tokens(init_idx)))

iter 1 the meaning of life is of of of to of in in in in in in of in of of of of of of of
iter 11 the meaning of life is of in in and in in in and and and and and and and and and and and and and
iter 21 the meaning of life is of in in and in in in and and and and and and and and and and and and and
iter 31 the meaning of life is of in in and in in in and and and and and and and and and and and and and
iter 41 the meaning of life is of in in and in in in and and and and and and and and and and and and and
iter 51 the meaning of life is of in in and in in in and and and and and and and and and and and and and
iter 61 the meaning of life is of in in and in in in and and and and and and and and and and and and and
iter 71 the meaning of life is of in in and in in in and and and and and and and and and and and and and


In [64]:
len(tokenizer.vocab)

30522